In [3]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import base64
import os
import secrets

class SecureCommunicationSystem:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Secure Communication System")
        self.root.geometry("600x500")

        self.selected_image_path = None
        self.selected_video_path = None
        self.salt = None
        self.key = None
        self.cipher_suite = None

        self.setup_ui()

    def derive_key(self, password, salt=None):
        """Generate a secure key using PBKDF2"""
        if not salt:
            salt = secrets.token_bytes(16)

        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=salt,
            iterations=100000,
        )
        key = base64.urlsafe_b64encode(kdf.derive(password.encode()))
        return salt, key

    def initialize_encryption(self, for_extraction=False, stored_salt=None):
        """Initialize encryption with a password"""
        password = simpledialog.askstring("Password",
                                            "Enter password to " +
                                            ("decrypt" if for_extraction else "encrypt") +
                                            " the message:",
                                            show='*')
        if password:
            if for_extraction and stored_salt:
                self.salt, self.key = self.derive_key(password, stored_salt)
            else:
                self.salt, self.key = self.derive_key(password)
            self.cipher_suite = Fernet(self.key)
            return True
        return False

    def select_image(self):
        """Select an image file"""
        self.selected_image_path = filedialog.askopenfilename(
            filetypes=[("Image files", ".png;.jpg;*.jpeg")])
        if self.selected_image_path:
            messagebox.showinfo("Success", "Image selected successfully")
            # Reset encryption state when new image is selected
            self.salt = None
            self.key = None
            self.cipher_suite = None

    def hide_message_in_image(self):
        """Hide encrypted message in image using LSB steganography"""
        if not self.selected_image_path:
            messagebox.showerror("Error", "Please select an image first")
            return

        if not self.cipher_suite and not self.initialize_encryption():
            return

        message = simpledialog.askstring("Input", "Enter your message:")
        if not message:
            return

        try:
            # Encrypt the message
            encrypted_data = self.cipher_suite.encrypt(message.encode())

            # Add salt to the encrypted data
            final_data = self.salt + encrypted_data

            # Convert to binary
            binary_data = ''.join(format(byte, '08b') for byte in final_data)

            # Load and prepare image
            img = Image.open(self.selected_image_path)
            pixels = list(img.getdata())
            width, height = img.size

            if len(binary_data) > len(pixels) * 3:
                raise ValueError("Message too large for this image")

            # Embed data
            binary_data += '11111110'  # End marker
            data_index = 0
            new_pixels = []

            for pixel in pixels:
                new_pixel = list(pixel)
                for color_channel in range(3):
                    if data_index < len(binary_data):
                        new_pixel[color_channel] = (new_pixel[color_channel] & ~1) | int(binary_data[data_index])
                        data_index += 1
                new_pixels.append(tuple(new_pixel))

                if data_index >= len(binary_data):
                    break

            # Save new image
            new_image = Image.new(img.mode, img.size)
            new_image.putdata(new_pixels)

            save_path = filedialog.asksaveasfilename(
                defaultextension=".png",
                filetypes=[("PNG files", "*.png")])

            if save_path:
                new_image.save(save_path, 'PNG')
                messagebox.showinfo("Success", "Message hidden successfully")

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    def extract_message_from_image(self):
        """Extract and decrypt hidden message from image"""
        if not self.selected_image_path:
            messagebox.showerror("Error", "Please select an image first")
            return

        try:
            # Load image and extract binary data
            img = Image.open(self.selected_image_path)
            pixels = list(img.getdata())

            binary_data = ''
            for pixel in pixels:
                for color_value in pixel[:3]:
                    binary_data += str(color_value & 1)

            # Find end marker and extract data
            delimiter_index = binary_data.find('11111110')
            if delimiter_index == -1:
                raise ValueError("No hidden message found")

            binary_data = binary_data[:delimiter_index]

            # Convert binary to bytes
            byte_data = bytearray()
            for i in range(0, len(binary_data), 8):
                byte = binary_data[i:i+8]
                byte_data.append(int(byte, 2))

            # Extract salt from the hidden data and ensure it's in bytes
            stored_salt = bytes(byte_data[:16])
            encrypted_data = byte_data[16:]

            # Initialize decryption with the stored salt
            if not self.initialize_encryption(for_extraction=True, stored_salt=stored_salt):
                return

            # Attempt decryption
            try:
                decrypted_message = self.cipher_suite.decrypt(bytes(encrypted_data))
                messagebox.showinfo("Success", f"Extracted message: {decrypted_message.decode()}")
            except Exception:
                messagebox.showerror("Error", "Invalid password or corrupted data")

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    def setup_ui(self):
        """Setup the user interface"""
        frame = tk.Frame(self.root, padx=20, pady=20)
        frame.pack(expand=True, fill='both')

        # Create sections
        image_section = tk.LabelFrame(frame, text="Image Steganography", padx=10, pady=10)
        image_section.pack(fill='x', pady=10)

        # Image buttons
        tk.Button(image_section, text="Select Image",
                    command=self.select_image).pack(pady=5)
        tk.Button(image_section, text="Hide Message",
                    command=self.hide_message_in_image).pack(pady=5)
        tk.Button(image_section, text="Extract Message",
                    command=self.extract_message_from_image).pack(pady=5)

        # Status bar
        status_frame = tk.Frame(self.root)
        status_frame.pack(side='bottom', fill='x')
        tk.Label(status_frame, text="Ready", bd=1, relief=tk.SUNKEN, anchor=tk.W).pack(fill='x')

    def run(self):
        """Start the application"""
        self.root.mainloop()

if __name__ == "__main__":
    app = SecureCommunicationSystem()
    app.run()